In [1]:
%pip install scipy
%pip install numpy
%pip install pandas
%pip install networkx
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import networkx as nx
from scipy import sparse
from scipy.sparse.linalg import norm
from sklearn.model_selection import train_test_split
from typing import Tuple, Mapping, Any

In [3]:
df = pd.read_csv("res\\rating.csv")

In [4]:
df['rating'].describe()

count    2.000026e+07
mean     3.525529e+00
std      1.051989e+00
min      5.000000e-01
25%      3.000000e+00
50%      3.500000e+00
75%      4.000000e+00
max      5.000000e+00
Name: rating, dtype: float64

In [5]:
df['userId'] = df['userId'] - 1
df['movieId'] = df['movieId'] + max(df['userId']) - 1

In [6]:
df.head()

,userId,movieId,rating,timestamp
0,0,138493,3.5,2005-04-02 23:53:47
1,0,138520,3.5,2005-04-02 23:31:16
2,0,138523,3.5,2005-04-02 23:33:39
3,0,138538,3.5,2005-04-02 23:32:07
4,0,138541,3.5,2005-04-02 23:29:40


In [7]:
def split_by_target_column_value(df: pd.DataFrame, test_size: float, target_column: str, target_value: Any,
                                 random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Разделяет DataFrame на обучающий и тестовый наборы на основе определенного значения целевого столбца.
    Выдает в качестве тестового набора test_size от набора, в котором значение в target_column == target_value
    В тренировочном все остальное
    Параметры:
        df (pd.DataFrame): DataFrame для разделения.
        test_size: Пропорция данных, которые следует включить в тестовый набор.
        target_column (str): Название целевого столбца для разделения.
        target_value (Any): Значение целевого столбца, используемое для разделения.
        random_state (int): Зерно для генератора случайных чисел.

    Возвращает:
        Tuple[pd.DataFrame, pd.Series, pd.DataFrame, pd.Series]: Данные для обучения; тестовые данные для целевой колонки, которые не попали в data,
        тренировочные данные для целевой колоники, которые попали в data
    """
    grouped = df.groupby(by=target_column)
    train_df, test_df = train_test_split(grouped.get_group(target_value), test_size=test_size,
                                         random_state=random_state)
    data = pd.concat([group for ind, group in grouped if ind != target_value], ignore_index=True)
    data = pd.concat([train_df, data], ignore_index=True)
    return data, test_df, train_df


In [8]:
data, test1, train1 = split_by_target_column_value(df, test_size=0.5, target_column='userId', target_value=1)

In [9]:
data.head()

,userId,movieId,rating,timestamp
0,1,140945,4.0,2000-11-21 15:35:16
1,1,140239,5.0,2000-11-21 15:36:54
2,1,140071,4.0,2000-11-21 15:32:28
3,1,141650,3.0,2000-11-21 15:29:58
4,1,139382,2.0,2000-11-21 15:36:09


In [10]:
test1.head()

,userId,movieId,rating,timestamp
175,1,138494,4.0,2000-11-21 15:34:49
180,1,138751,5.0,2000-11-21 15:36:54
221,1,142004,5.0,2000-11-21 15:30:59
206,1,140463,2.0,2000-11-21 15:36:09
188,1,139415,5.0,2000-11-21 15:36:54


In [11]:
train1.head()

,userId,movieId,rating,timestamp
211,1,140945,4.0,2000-11-21 15:35:16
201,1,140239,5.0,2000-11-21 15:36:54
199,1,140071,4.0,2000-11-21 15:32:28
218,1,141650,3.0,2000-11-21 15:29:58
186,1,139382,2.0,2000-11-21 15:36:09


In [12]:
from abc import ABC, abstractmethod
        
# https://github.com/danieljunhee/Tutorial-on-Personalized-PageRank/blob/master/Personalized_PageRank_Tutorial.ipynb
class PageRank():
    
    def __init__(self, a: float = 0.8, b: float = 0.15, c: float=0.05) -> None:
        # Decide values of a,b,c. For this tutorial, let's use a = 0.8, b = 0.15, c = 0.05.
        self.a = a
        self.b = b
        self.c = c
        
    def get_pagerank(self, transition_matrix: sparse.csr_matrix, user_person_vec: sparse.csr_matrix) -> np.ndarray:
        
        # Initialize PageRank vector: height x 1 vector of all 1/height 's
        # We also represent the PageRank vector as a sparse csr matrix so that every arithmetic operation result
        # is stored as a sparse csr matrix throughout the algorithm... again dealing with sparsity issue & memory issue
        
        height = transition_matrix.shape[0]
        user_pagerank_vec = sparse.csr_matrix(np.ones((height,1)) / height)

        # Update the PageRank vector until convergence! Our convergence criterion is to see whether
        # the magnitude of the difference of the PageRank vector after an update is 0
        converged = False
        while (not converged):
            # (deep) copy the current PageRank vector to compare with the updated vector
            old_user_pagerank_vec = user_pagerank_vec.copy()
            
            # Update!
            user_pagerank_vec = self.a*transition_matrix.dot(user_pagerank_vec) \
                                + self.b*sparse.csr_matrix(user_person_vec.dot(user_pagerank_vec)[0,0]*np.ones((height,1))) \
                                + self.c*sparse.csr_matrix((1/height)*np.ones((height,1)))
                    
            # Compare with the PageRank vector before the update. Check the magnitude of the difference of the vector
            print("magnitude of difference vector:", norm(user_pagerank_vec-old_user_pagerank_vec))
            
            if norm(user_pagerank_vec-old_user_pagerank_vec) == 0:
                converged = True
        return user_pagerank_vec.toarray()
    
    

In [35]:
def create_transition_matrix_from_pd(data: pd.DataFrame,
               source: str, target: str, edge_attr: str) -> sparse.csr_matrix:
    
    obj_source_to_N = {}
    obj_target_to_N = {}
    N_to_obj = {}
    
    source_uniq = set(data[source])
    for index, obj in zip(range(len(source_uniq)), source_uniq):
        obj_source_to_N[obj] = index
        N_to_obj[index] = obj
        
    
    target_uniq = set(data[target])
    for index, obj in zip(range(len(source_uniq), len(source_uniq) + len(target_uniq)), target_uniq):
        obj_target_to_N[obj] = index
        N_to_obj[index] = obj
    
    count_top = len(N_to_obj)
    transition_matrix = sparse.lil_matrix((count_top, count_top))

    def process_edge(row):
        from_top = obj_source_to_N[row[source]]
        to_top = obj_target_to_N[row[target]]
        transition_matrix[from_top, to_top] = row[edge_attr]
        transition_matrix[to_top, from_top] = row[edge_attr]
        


    data.apply(process_edge, axis=1)
    print(f"Обработано все ребра")
        
    print("Создана матрица, идет нормирование...")
    # Получаем суммы элементов каждой строки в виде numpy массива
    
    transition_matrix_csr = sparse.csr_matrix(transition_matrix)
    del transition_matrix
    
    row_sums = transition_matrix_csr.sum(axis=1)
    transition_matrix_csr /= row_sums
    print("Сумма посчитана")
        
    return transition_matrix_csr, N_to_obj
        
data_test = pd.DataFrame({
    'source': ['A', 'A', 'B', 'B', 'C'],
    'target': ['X', 'Y', 'X', 'Z', 'Z'],
    'edge_attr': [4, 5, 4, 5, 3]
})  

matrix, N_to_obj = create_transition_matrix_from_pd(data_test, 'source', 'target', 'edge_attr')

create_transition_matrix_from_pd(data, 'userId', 'movieId', 'rating')

# Проверяем результаты
print("Матрица переходов:")
print(pd.DataFrame(matrix.toarray()))

print("\nОтображение индексов:")
print(N_to_obj)


Обработано все ребра
Создана матрица, идет нормирование...
Сумма посчитана


In [ ]:
data['userId'].min()

0

In [ ]:
data['movieId'].value_counts()

movieId
138787    67310
138847    66172
138809    63366
139084    63299
138971    59714
          ...  
264036        1
217364        1
251398        1
251400        1
249001        1
Name: count, Length: 26744, dtype: int64

In [ ]:
def create_transition_matrix_from_pd(data: pd.DataFrame,
               source: str, target: str, edge_attr: str) -> sparse.csr_matrix:
    
    count_top = data[source].nunique() + data[target].nunique()
    transition_matrix = sparse.lil_matrix((count_top, count_top))
        
   # Заполнение матрицы значениями ребер и их весами
    for _, row in data.iterrows():
        source, target, weight = row[source], row[target], row[edge_attr]
        transition_matrix[source, target] = weight
        transition_matrix[target, source] = weight
    
    return transition_matrix.tocsr()

In [ ]:
matrix, N_to_obj = create_transition_matrix_from_pd(data, 'userId', 'movieId', 'rating')

C:\Users\odo20\AppData\Local\Temp\ipykernel_15688\391877748.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  source, target, weight = row[source], row[target], row[edge_attr]


IndexError: index 140945 is out of bounds for axis 0 with size 4

In [ ]:
pg = PageRank()
pagerank = pg.get_pagerank(matrix, matrix[0])
pagerank